In [152]:
import requests, json, regex, js2xml, re
from bs4 import BeautifulSoup, Comment


def get_crimes(data):
    crimes=["Anti-social behaviour","Bicycle theft","Burglary","Criminal damage and arson","Drugs","Other theft","Possession of weapons","Public order","Robbery","Shoplifting","Theft from the person","Vehicle crime","Violence and sexual offences","Other crime"]
    string = data[0]
    pattern= re.compile('\[[^)]+\]')
    string = string.strip()

    split = string.split('data: ',1)[1]
    raw_data= pattern.findall(split)[0]
    all_shit = []
    for i in raw_data.split('\n'):
        res = i.strip('][').split(', ')
        all_shit.append(res)

    numbers = []
    for i in all_shit:
        for word in i[-1].split('],'):
            if word.isdigit():
                numbers.append(int(word))

#     print(list(zip(crimes,numbers)))
#     print(sum(numbers))
    return list(zip(crimes,numbers)), sum(numbers)

def get_crime_data(postcode):
    postcode = postcode.replace(" ","%20")
    URL = 'https://www.crime-statistics.co.uk/postcode/'+postcode
    print("Getting Crime Data")
    page = requests.get(URL)
    pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')

    print("Parsing Response")
    soup = BeautifulSoup(page.content, 'html.parser')
    element = soup.find("div", id="divPieChart")
    script = element.find('script')

    print("Extracting Data")
    data = pattern.findall(str(script))

#     print ("getting crime statistics")
    return get_crimes(data)


In [175]:
import os
import glob
from difflib import SequenceMatcher

def get_maps(city):
    maps = []
    city = city + "*"
    for file in glob.glob(city):
        maps.append(file)
    return maps

def get_locations(city):
    print("Getting Map Files")
    maps = get_maps(city)
    longs = []
    lats = []
#     word_file = open("words.txt", "a")
    print("Processing Map Files")
    for f in maps:
        ofile = open(f, 'r')
        for line in ofile:
            lat, long, blank, prob1, word, prob2 = line.split(',')
            word=word.replace('"','')
            p_chimney = SequenceMatcher(None, word, "chimney").ratio()
            p_chy = SequenceMatcher(None, word, "chy").ratio()
            if p_chimney > p_chy:
                p_use = p_chimney
            else:
                p_use = p_chy
            if(p_use > 0.8):
                longs.append(long)
                lats.append(lat)
    locations = set(zip(longs,lats))
#     word_file.close()
    return locations

def get_postcode(city):
    print("Getting Chimney Locations")
    locations = get_locations(city)
#     print(locations)
    postcodes = []
    print("Getting Postcodes from Lat/Long")
    for location in locations:
        long, lat = location
        URL = 'http://api.postcodes.io/postcodes?lon='+long+'&lat='+lat
        page = requests.get(URL)
        json_obj = json.loads(page.content)
        if json_obj["result"]:
            pc = json_obj["result"][0]['postcode']
#             postcodes.append(pc)
            postcodes.append((pc, location))
    print("Returning Unique Postcodes")
    return set(postcodes)


In [178]:
def crimes_by_city(city):
    print("Getting Postcodes of Chimneys in: " + city)
    postcodes = get_postcode(city)
    for pc in postcodes:
#         print("Getting Crime Data")
        breakdown, total = get_crime_data(pc[0])
        print()
        print("###########################################")
        print("POSTCODE: "+pc[0])
        print("CRIME BREAKDOWN:" + str(breakdown))
        print()
        print("TOTAL CRIMES = "+str(total))
        print("############################################")

In [180]:
crimes_by_city("Burnley")

Getting Postcodes of Chimneys in: Burnley
Getting Chimney Locations
Getting Map Files
Processing Map Files
Getting Postcodes from Lat/Long
Returning Unique Postcodes
Getting Crime Data
Parsing Response
Extracting Data

###########################################
POSTCODE: BB11 1NL
CRIME BREAKDOWN:[('Anti-social behaviour', 235), ('Bicycle theft', 2), ('Burglary', 22), ('Criminal damage and arson', 68), ('Drugs', 6), ('Other theft', 50), ('Possession of weapons', 4), ('Public order', 33), ('Robbery', 3), ('Shoplifting', 51), ('Theft from the person', 2), ('Vehicle crime', 26), ('Violence and sexual offences', 204), ('Other crime', 12)]

TOTAL CRIMES = 718
############################################
Getting Crime Data


KeyboardInterrupt: 